## 5.9 자연어 처리 방법에 대한 추천 시스템 응용
- 자연어 처리를 응용하여 콘텐츠 기반 추천과 비슷한 상품을 찾을 수 있음
  - LDA(토픽 모델), word2vec 등을 활용
- 위 방법을 사용자 행동 이력에 적용하여 협업 필터링 기반 추천도 가능

### 토픽 모델
- 단어가 가진 의미는 문장 주제(토픽)에 따라 사용되는 빈도가 다름
- 하나의 문장은 여러 토픽으로 구성되며 여러 주제가 하나의 문장을 구성
- **LDA(Latent Dirichlet Allocation)**
  - 토픽 할당에 Dirichlet 분포를 사전 분포로 가정해 베이즈 추정을 한 모델
    1. 전처리1: 문장을 단어로 분해 (형태소 분석 라이브러리 사용)
    2. 전처리2: 명사와 형용사만 (혹은 명사만) 남김
    3. 데이터를 LDA에 입력하여 아래와 같은 분포를 계산
       - 각 토픽별 단어 분포
       - 문장 토픽 분포
    4. LDA 데이터 입력시에는 토픽의 갯수 파라미터를 정해두어야 함
  - LDA는 비지도 학습이며 각 토픽이 어느 분포에 가까운지 정하여 주제를 추정함
    - 예) 토픽 1에서는 '축구'라는 단어가 0.3의 확률로 생성되기 쉽고, 토픽 2에서는 '선수'라는 단어는 0.001의 확률로 생성되기 쉬움

- MoveLens의 태그와 장르 정보에 LDA를 적용해 콘텐츠 기반 추천을 진행

In [1]:
# 부모 폴더의 경로 추가
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [2]:
# Movielens 데이터 로딩
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='./data/ml-10M100K/')
movielens = data_loader.load()

In [ ]:
# !pip install gensim

In [3]:
import gensim
import logging
from gensim.corpora.dictionary import Dictionary

movie_content = movielens.item_content.copy()
# tag가 부여되어 있지 않은 영화도 있지만, genre는 모든 영화에 부여되어 있다
# tag와 genre를 결합한 것을 영화 콘텐츠 정보로 해서 비슷한 영화를 찾아 추천한다
# tag가 없는 영화는 NaN으로 되어 있으므로, 빈 리스트로 변환한 뒤 처리한다
movie_content['tag_genre'] = movie_content['tag'].fillna("").apply(list) + movie_content['genre'].apply(list)
movie_content['tag_genre'] = movie_content['tag_genre'].apply(lambda x:list(map(str, x)))

# 태그와 장르 데이터를 사용해 LDA를 학습한다
tag_genre_data = movie_content.tag_genre.tolist()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
common_dictionary = Dictionary(tag_genre_data)
common_corpus = [common_dictionary.doc2bow(text) for text in tag_genre_data]

# LDA 학습
lda_model = gensim.models.LdaModel(common_corpus, id2word=common_dictionary, num_topics=50, passes=30)
lda_topics = lda_model[common_corpus]

2024-02-26 21:43:59,177 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-02-26 21:43:59,307 : INFO : adding document #10000 to Dictionary<14749 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...>
2024-02-26 21:43:59,313 : INFO : built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)
2024-02-26 21:43:59,315 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)", 'datetime': '2024-02-26T21:43:59.315005', 'gensim': '4.3.2', 'python': '3.11.3 (main, Dec 16 2023, 08:37:08) [Clang 15.0.0 (clang-1500.0.40.1)]', 'platform': 'macOS-14.0-x86_64-i386-64bit', 'event': 'created'}
2024-02-26 21:43:59,410 : INFO : using symmetric alpha at 0.02
2024-02-26 21:43:59,410 : INFO : using symmetric eta at 0.02
2024-02-26 21:43:59,413 :

In [4]:
# LDAContent추천
from src.lda_content import LDAContentRecommender
recommender = LDAContentRecommender()
recommend_result = recommender.recommend(movielens)

2024-02-26 21:47:48,089 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-02-26 21:47:48,205 : INFO : adding document #10000 to Dictionary<14749 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...>
2024-02-26 21:47:48,211 : INFO : built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)
2024-02-26 21:47:48,212 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)", 'datetime': '2024-02-26T21:47:48.212971', 'gensim': '4.3.2', 'python': '3.11.3 (main, Dec 16 2023, 08:37:08) [Clang 15.0.0 (clang-1500.0.40.1)]', 'platform': 'macOS-14.0-x86_64-i386-64bit', 'event': 'created'}
2024-02-26 21:47:48,301 : INFO : using symmetric alpha at 0.02
2024-02-26 21:47:48,303 : INFO : using symmetric eta at 0.02
2024-02-26 21:47:48,305 :

- LDA 학습이 완료되면 토픽에 나타나기 쉬운 단어 확인 가능
  - 예) 디즈니나 애니메이션이라는 단어가 많이 나오므로 애니메이션 토픽임을 확인 가능

### 5.9.2 LDA를 사용한 콘텐츠 기반 추천
- LDA를 사용하여 텍스트로 컨텐츠를 분류하고 유사도를 측정할 수 있음
- MovieLens에서는 각 영화에 가장 큰 점수를 가진 주제를 할당
  - 예) 토픽 1: 토이스토리, 토픽 5: 타이타닉 등
- 사용자가 최근 가장 높이 평가한 10개의 토픽을 확인하여 콘텐츠를 추천하고 노출되지 않은 10개의 영화 추천
- 코드는 src.lda_content.py에서 확인
- 결과는 Precision@K=0.004, Recall@K=0.012

In [5]:
# 평가
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.004, Recall@K=0.014


### 5.9.3 LDA를 사용한 협업 필터링 추천
- 사용자의 구입 이력이나 열람 이력은 리스트 형태로 표현 가능
  - User1:[item1, item2, item23, item4]
  - User2:[item52, item3, item1, item8]
- 각 아이템을 단어로 보고 사용자가 행동한 아이템의 집합을 문장으로 취급하여 LDA를 적용 가능
- MovieLens 데이터로 추천하면 각 사용자별로 가장 큰 토픽을 추출하고 해당 토픽에서 나오기 쉬우면서도 해당 사용자가 아직 평가하지 않은 영화를 상위부터 10개 추천

In [6]:
# LDACollaboration 추천
from src.lda_collaboration import LDACollaborationRecommender
recommender = LDACollaborationRecommender()
recommend_result = recommender.recommend(movielens)

2024-02-26 21:54:37,760 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-02-26 21:54:37,817 : INFO : built Dictionary<4983 unique tokens: ['185', '231', '292', '316', '329']...> from 997 documents (total 67749 corpus positions)
2024-02-26 21:54:37,818 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<4983 unique tokens: ['185', '231', '292', '316', '329']...> from 997 documents (total 67749 corpus positions)", 'datetime': '2024-02-26T21:54:37.818229', 'gensim': '4.3.2', 'python': '3.11.3 (main, Dec 16 2023, 08:37:08) [Clang 15.0.0 (clang-1500.0.40.1)]', 'platform': 'macOS-14.0-x86_64-i386-64bit', 'event': 'created'}
2024-02-26 21:54:37,864 : INFO : using symmetric alpha at 0.02
2024-02-26 21:54:37,865 : INFO : using symmetric eta at 0.02
2024-02-26 21:54:37,865 : INFO : using serial LDA version on this node
2024-02-26 21:54:37,885 : INFO : running online (multi-pass) LDA training, 50 topics, 30 passes over the supplied corpus of 997 documents, updating mo

In [7]:
# 평가
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.020, Recall@K=0.065


- Precisioin@K=0.024, Recall@k=0.075의 값이 나옴 (책에서는)
- 행동이력에 LDA를 적용하면 EDA를 하는데도 도움이 됨 (많이 사용되는 단어 빈도를 알 수 있으므로)

### 5.9.4 word2vec
- **분포 가설** : 단어의 의미는 주변 단어에 따라 결정됨을 의미
  - 서점에서 구매한 OO는 재미있다, OO : 책, 만화, 서적 등이 될 수 있음
- **word2vec** : 분포 가설을 기반으로 단어 의미를 표현한 방법
  - 예측 확률도 잘나오고 구현도 쉬워 자주 사용됨
- word2vec을 학습시키면 각 단어의 벡터를 알 수 있음. 벡터로 유사도를 설계 가능

### 5.9.5 word2vec을 사용한 콘텐츠 기반 추천 
- 온라인 쇼핑 사이트의 서적 추천을 예시로 들어 설명
  - 서적 줄거리의 단어 벡터의 평균을 해당 서적의 벡터로 간주 -> 서적 벡터 유사도로 아이템 추천
  - 자주 나오는 단어와 전문 용어가 동등하게 추천되는 방법을 해결하기 위해 아래와 같은 방법 사용
    - td-idf로 해당 문제에 특징있는 단어만 추출
    - td-idf 가중치를 사용해 벡터를 계산
    - SWEM(각 차원의 최대, 최소값을 추출한 최대, 최소 벡터를 결합하여 문장 벡터로 간주하는 방법)
- word2vec을 발전시켜 doc2vec이라는 방법 사용
  - 단어 뿐 아니라 문장 자체에도 벡터를 부여
  - 하이퍼파라미터를 적절하게 조정하면 word2vec 보다 높은 성능을 발휘
  - 실무에서는 word2vec으로 실시간 추천을 사용하거나 파라미터 조정으로 빠르게 적용 가능하므로 word2vec 추천 결과도 함께 확인하는 것을 추천

In [8]:
movie_content = movielens.item_content.copy()

# tag가 부여되어 있지 않은 영화가 있지만. genre는 모든 영화에 부여되어 있다
# tag와 genre를 결합한 것을 영화의 콘텐츠 정보로 해서 비슷한 영화를 찾아서 추천한다
# tag가 없는 영화에서는 NaN으로 되어 있으므로, 빈 리스트로 초기화한다
movie_content['tag_genre'] = movie_content['tag'].fillna("").apply(list) + movie_content['genre'].apply(list)
movie_content['tag_genre'] = movie_content['tag_genre'].apply(lambda x:set(map(str, x)))

# 태그와 장르 데이터를 사용해서, word2vec를 학습한다
tag_genre_data = movie_content.tag_genre.tolist()
model = gensim.models.word2vec.Word2Vec(tag_genre_data, vector_size=100, window=100, sg=1, hs=0, epochs=50, min_count=5)

2024-02-26 22:15:53,661 : INFO : collecting all words and their counts
2024-02-26 22:15:53,662 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-02-26 22:15:53,689 : INFO : PROGRESS: at sentence #10000, processed 86882 words, keeping 14749 word types
2024-02-26 22:15:53,692 : INFO : collected 15261 word types from a corpus of 91361 raw words and 10681 sentences
2024-02-26 22:15:53,693 : INFO : Creating a fresh vocabulary
2024-02-26 22:15:53,704 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 2417 unique words (15.84% of original 15261, drops 12844)', 'datetime': '2024-02-26T22:15:53.704337', 'gensim': '4.3.2', 'python': '3.11.3 (main, Dec 16 2023, 08:37:08) [Clang 15.0.0 (clang-1500.0.40.1)]', 'platform': 'macOS-14.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2024-02-26 22:15:53,705 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 73012 word corpus (79.92% of original 91361, drops 18349)', 'datetime': '202

In [9]:
# 에니메이션 태그를 입력하여 결과를 확인한다
# anime 태그와 비슷한 태그를 확인한다
model.wv.most_similar('anime')

[('zibri studio', 0.8007268309593201),
 ('studio ghibli', 0.796676754951477),
 ('hayao miyazaki', 0.7703091502189636),
 ('pelicula anime', 0.7695381045341492),
 ('miyazaki', 0.768201470375061),
 ('japan', 0.6450369954109192),
 ('Animation', 0.6024659872055054),
 ('wilderness', 0.49096623063087463),
 ('animation', 0.4680529832839966),
 ('steampunk', 0.4642772376537323)]

- 최근 가장 높게 평가한 영화 5개의 영화 벡터 평균으로 사용자의 벡터와 코사인 거리가 가까운 영화를 찾아서 추천

In [10]:
# Word2vecContent 추천
from src.word2vec import Word2vecRecommender
recommender = Word2vecRecommender()
recommend_result = recommender.recommend(movielens)

2024-02-26 22:20:53,895 : INFO : collecting all words and their counts
2024-02-26 22:20:53,897 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-02-26 22:20:53,923 : INFO : PROGRESS: at sentence #10000, processed 86882 words, keeping 14749 word types
2024-02-26 22:20:53,926 : INFO : collected 15261 word types from a corpus of 91361 raw words and 10681 sentences
2024-02-26 22:20:53,927 : INFO : Creating a fresh vocabulary
2024-02-26 22:20:53,939 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 2417 unique words (15.84% of original 15261, drops 12844)', 'datetime': '2024-02-26T22:20:53.939719', 'gensim': '4.3.2', 'python': '3.11.3 (main, Dec 16 2023, 08:37:08) [Clang 15.0.0 (clang-1500.0.40.1)]', 'platform': 'macOS-14.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2024-02-26 22:20:53,940 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 73012 word corpus (79.92% of original 91361, drops 18349)', 'datetime': '202

In [11]:
# 평가
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.010, Recall@K=0.033


### 5.9.6 word2vec을 사용한 협업 필터링 추천(item2vec)
- 이 방법을 item2vec, prod2vec이라고도 하며 추천 성능이 높아 야후, 에어비엔비에서도 사용
- 사용자 행동 이력을 단어 집합으로 간주하고 word2vec 적용
  - 사용자가 행동한 순서대로 아이템을 나열하는 것이 중요
  - word2vec에는 window_size 파라미터가 있어 액션 순서까지 고려하여 학습시킴

- User1:[item1, item2, item23, item4]
  User2:[item52, item3, item1, item8]
- 학습을 완료하면 각 단어에 대해 아래와 같은 벡터를 얻을 수 있음
  - item1:[0.3, 0.1, 0.6...]
  - item2:[0.1, 0.9, 0.2...]
- 위 벡터로 아이템 간 유사도를 계산하여 관련 아이템 추천을 구현
  - 사용자가 item5, item9 구매 시 사용자의 벡터를 item5와 item9의 평균 벡터로 표현하고 해당 벡터와 가까운 아이템을 추천
  - 사용자 벡터 표현 방법들
    - 가장 최근 아이템 몇 개를 사용해 해당 사용자를 표현하는 관점 (오래된 벡터는 최근 선호도 표현 어려움)
    - 최근 행동한 아이템에 큰 가중치를 부여하여 벡터를 설계
- 아이템 벡터를 유지할 수 있으면 사용자의 벡터는 해당 아이템 벡터의 사칙연산으로 실시간 온라인 추천 등에 사용

In [12]:
# Item2vecContent 추천
from src.item2vec import Item2vecRecommender
recommender = Item2vecRecommender()
recommend_result = recommender.recommend(movielens)

2024-02-26 22:35:21,072 : INFO : collecting all words and their counts
2024-02-26 22:35:21,073 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-02-26 22:35:21,081 : INFO : collected 4983 word types from a corpus of 67749 raw words and 997 sentences
2024-02-26 22:35:21,082 : INFO : Creating a fresh vocabulary
2024-02-26 22:35:21,090 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 2174 unique words (43.63% of original 4983, drops 2809)', 'datetime': '2024-02-26T22:35:21.090106', 'gensim': '4.3.2', 'python': '3.11.3 (main, Dec 16 2023, 08:37:08) [Clang 15.0.0 (clang-1500.0.40.1)]', 'platform': 'macOS-14.0-x86_64-i386-64bit', 'event': 'prepare_vocab'}
2024-02-26 22:35:21,091 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 62516 word corpus (92.28% of original 67749, drops 5233)', 'datetime': '2024-02-26T22:35:21.091217', 'gensim': '4.3.2', 'python': '3.11.3 (main, Dec 16 2023, 08:37:08) [Clang 15.0.0 (clang-150

In [13]:
# 평가
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.025, Recall@K=0.081


- 책에서 사용한 하이퍼파라미터는 콘텐츠 기반 추천에서 추출한 것을 튜닝함
- 하이퍼파라미터는 자사 데이터로 직접 튜닝하는 것이 좋음
- 위에서 소개한 방법 외에도 BERT 모델 등이 있다 (2018년에 좋은 성적을 거둔 모델)

## 5.10 딥러닝
- 딥러닝 추천 시스템 응용은 2015년 정도부터 시작

### 5.10.1 딥러닝을 활용한 추천 시스템
- 실무에서는 2가지 방법으로 딥러닝을 추천 시스템에 활용
  - 이미지나 문장 등 비구조 데이터의 (특질량)피처 추출기로 활용
  - 복잡한 사용자 행동과 아이템 피터 모델링
 
### 5.10.11 이미지나 문장 등 비구조 데이터의 특질량 추출기로 활용
- 이미지로 딥러닝 기법을 사용하여 각 이미지의 피처를 추출함
- 딥러닝 학습 모델을 문장에 사용하면 원시 데이터를 그대로 사용하는 것에 비해 유사성이 유추된 저차원 벡터를 얻을 수 있다
- 딥러닝을 활용하여 카테고리나 태그 정보가 아닌 아이템 콘텐츠 자체의 유사도를 계산해서 추천할 수 있음
  - 인스타그램의 경우는 비슷한 이미지를, 스포티파이에서는 비슷한 음악을 추천
  - 딥리닝을 활용하면 콜드 스타트 문제도 개선 가능

### 5.10.12 복잡한 사용자 행동과 아이템 특질량 모델링
- 추천 시스템에서 딥러닝의 장점으로 다음 2가지를 들 수 있음
  - 비선형 데이터 모델링
  - 시계열 데이터 모델링

#### 비선형 데이터 모델링
- 이전 절의 행렬 분해는 벡터와 내적을 사용한다는 점에서 선형 모델임
- 딥러닝은 비선형을 표현 가능

![NCF개념도](./images/img_5-17.png)

- 위 그림은 행렬 분해를 딥러닝화 한 NCF 개념도임
- 신경망의 여러 층으로 구성되어 있어 사용자 X 아이템의 복잡한 데이터 학습이 가능하고 기존 행렬 분해보다 높은 예측 정확도를 얻을 수 있음

![DeepFM](./images/img_5-18.png)

- DeepFM은 Factorization Machines를 딥러닝화한 방법
- 아이템이나 사용자의 특징량에 대해 특질량 엔지니어링이 필요 없으며 그대로 모델에 입력 가능

![Wide and Deep](./images/img_5-19.png)

- 구글에서 발표한 Wide and Deep 방법이 있음
- 위의 그림과 같이 Wide 부분과 Deep 부분의 2개로 구성
  - Wide 부분은 아이템이나 사용자의 특징량을 입력으로 하여 1층의 선형 모델을 거침
  - 오른쪽의 Deep 부분에서는 Embedding 층을 내장해 다층으로 함으로써 보다 일반화된 추상적인 표현을 얻을 수 있음
  - 이 2개를 조합하면 예측 정확도를 높이면서 다양하게 추천 가능
  - 이 알고리즘은 구글의 AI Platform에서 사용 가능
 
#### 시계열 데이터 모델링
- RNN이나 LSTM을 시작으로 하는 시계열 정보를 다루는 데 많은 방법이 제안됨
- RNN, LSTM은 자연어 처리 분야에서 제안된 방법임
- 아래는 자연어 처리 방법을 행동 이력 데이터에 적용한 추천 알고리즘
  ![자연어처리방법을행동이력데이터로적용한알고리즘](./images/tbl_5-4.png)

### 5.10.2 구현
- 딥러닝 추천 알고리즘을 구현한 파이썬 라이브러리 소개
  ![딥러닝추천알고리즘라이브러리목록1](./images/tbl_5-5-1.png)
  ![딥러닝추천알고리즘라이브러리목록2](./images/tbl_5-5-2.png)

### 5.10.3 실무에서의 딥러닝 활용
- '특징량 추출기로 활용'과 '예측 모델로 활용'의 2가지 방법 소개

#### 5.10.3.1 특징량 추출기로 활용
- 실무에서 사용할 떄는 학습 완료 모델을 찾아 이를 자사의 아이템에 적용하고 특징량을 추출
  - 이미지라면 이미지넷 등으로 학습한 모델을 사용 가능
- 문장인 경우 위키피디아 등의 데이터로 학습된 word2vec이나 BERT 모델 사용

#### 5.10.3.2 예측 모델로 활용
- 딥러닝 기법보다 단순한 k-nearest 추천 시스템을 하이퍼파라미터 튜닝한 것보다 결과가 나쁜 경우가 있음
  - 원인으로는 전처리, 하이퍼파라미터 튜닝, 실험 설정 등이 적절하지 않았을 가능성
- 추천 시스템 도입은 고전적인 방법을 검증한 뒤 딥러닝 사용을 검토하면 좋음

### 5.11 슬롯머신 알고리즘(밴딧 알고리즘)
- 탐색과 활용을 어느 정도 효율적으로 수행해야 서비스에서의 이익이 최대화될 것인가라는 주제는 추천 시스템 분야에서 활발하게 논의되고 있음
- 다중 슬롯머신 문제라는 주제 안에서 연구가 진행중
- 슬롯(팔)이란 테스트 대상 아이템이나 알고리즘을 가리키며 도박에서 여러 슬롯으로 플레이하며 어떤 머신이 잘 맞는지 파악하는 모습에서 유래됨
- 다중 슬롯머신의 목적은 크게 2가지
  - 누적 이익 최대화
  - 누적 후회 최소화
- 이를 통해 최적 슬롯을 찾아내려하며 기본적으로는 누적 이익 크기를 고려하지 않는 순수 탐색 문제임
- 다중 슬롯 머신 문제는 다양한 종류의 알고리즘이 고안됨. 아래는 대표적인 알고리즘
  ![대표적인 슬롯머신 알고리즘](./images/tbl_5-6.png)

- 추천 시스템에서 슬롯머신 알고리즘이 유용한 대표적인 경우는 아래와 같음
  - 콜드 스타트 문제 대응
  - 개인화
  - 광고 추천
- 개인화는 뉴스 애플리케이션에서 각 사용자의 콘텍스트에 맞춘 슬롯머신 알고리즘을 활용한 사례가 알려져 있음
- 슬롯머신 문제에 대한 알고리즘은 최근 기술 기업에서 특히 그 활용 사례가 증가하고 있음